# Data Downloading

In [1]:
import os
os.system('bash Download.sh')

256

# Data Preprocessing

In [2]:
import pandas as pd
import numpy as np
import os, pickle
from tqdm import tqdm_notebook as tqdm

In [4]:
dataset_dir = '../ExtraSensory.per_uuid_features_labels'
label_dir = '../ExtraSensory.per_uuid_original_labels'
data = pd.DataFrame()
for filename in tqdm(os.listdir(dataset_dir)):
    key = filename[:filename.find('.')]
    csv = pd.read_csv(os.path.join(dataset_dir, filename), index_col=0)
    label = pd.read_csv(os.path.join(label_dir, key+'.original_labels.csv.gz'), index_col=0)
    csv = csv.join(label.drop('label_source', axis=1))
    csv = pd.concat([csv], keys=[key], names=['filename'])
    data = csv if data.empty else data.append(csv)

In [5]:
data.columns = pd.MultiIndex.from_tuples([tuple(i.split(':')) for i in data.columns], names=['1', '2', '3'])
labels_col = [i for i in data.columns if i[0].startswith('original_label')]
main_labels_col = [('main_label', i[1], i[2]) for i in labels_col[:7]]
df = pd.DataFrame(data[labels_col[:7]].values,\
                  columns=pd.MultiIndex.from_tuples(main_labels_col, names=['1', '2', '3']), \
                  index=data.index)

data = data.join(df)
data[('main_label', 'missing', np.nan)] = 1-data['main_label'].sum(axis=1)

In [8]:
print('{} data, {} columns'.format(*data.shape))
display(data.head())

377346 data, 401 columns


1                                                       raw_acc            \
2                                               magnitude_stats             
3                                                          mean       std   
filename                             timestamp                              
81536B0A-8DBF-4D8A-AC24-9543E2E4C8E0 1446141691        1.009726  0.002838   
                                     1446141752        1.009822  0.004624   
                                     1446141805        1.009667  0.004781   
                                     1446141873        1.008839  0.003543   
                                     1446141925        1.008193  0.001753   

1                                                                    \
2                                                                     
3                                                 moment3   moment4   
filename                             timestamp                        
81536B0A-8DBF-4D8A-AC24-9543E2E4C8E0 1446141691 -0.002296  0.005568   
                                     1446141752  0.003040  0.008459   
                                     1446141805 -0.007802  0.014457   
                                     1446141873  0.001831  0.007082   
                                     1446141925 -0.000744  0.002439   

1                                                                          \
2                                                                           
3                                               percentile25 percentile50   
filename                             timestamp                              
81536B0A-8DBF-4D8A-AC24-9543E2E4C8E0 1446141691     1.008208     1.009735   
                                     1446141752     1.007704     1.009619   
                                     1446141805     1.008038     1.009772   
                                     1446141873     1.007134     1.008803   
                                     1446141925     1.007142     1.008234   

1                                                                           \
2                                                                            
3                                               percentile75 value_entropy   
filename                             timestamp                               
81536B0A-8DBF-4D8A-AC24-9543E2E4C8E0 1446141691     1.011174      1.572784   
                                     1446141752     1.011857      1.754729   
                                     1446141805     1.011139      1.012852   
                                     1446141873     1.010433      1.511878   
                                     1446141925     1.009350      2.347186   

1                                                             \
2                                                              
3                                               time_entropy   
filename                             timestamp                 
81536B0A-8DBF-4D8A-AC24-9543E2E4C8E0 1446141691     6.684608   
                                     1446141752     6.684601   
                                     1446141805     6.684600   
                                     1446141873     6.684606   
                                     1446141925     6.684610   

1                                                                    ...    \
2                                               magnitude_spectrum   ...     
3                                                 log_energy_band0   ...     
filename                             timestamp                       ...     
81536B0A-8DBF-4D8A-AC24-9543E2E4C8E0 1446141691           5.043602   ...     
                                     1446141752           5.043674   ...     
                                     1446141805           5.043693   ...     
                                     1446141873           5.043562   ...     
                                     1446141925           5.043546  

In [9]:
users = list(set(data.index.get_level_values(0)))
np.random.shuffle(users)
train_user_num = int(len(users)*0.8)
valid_user_num = int(len(users)*0.1)
test_user_num = len(users) - train_user_num - valid_user_num
train, valid = data.loc[users[:train_user_num]], data.loc[users[train_user_num:train_user_num+valid_user_num]]
test = data.loc[users[-test_user_num:]]

# Train Network

In [13]:
import matplotlib.pyplot as plt
from keras.models import Sequential
import keras.layers
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import tensorflow as tf

Using TensorFlow backend.


In [10]:
train = train.fillna(0)
valid = valid.fillna(0)
test = test.fillna(0)

In [11]:
x_train = train[['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']]
y_train = train.iloc[:,-8:]
x_valid = valid[['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']]
y_valid = np.asarray(valid.iloc[:,-8:])
x_test = test[['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']]
y_test = np.asarray(test.iloc[:,-8:])

In [14]:
scaler = MinMaxScaler() #scale features between 0 and 1
x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)
x_test = scaler.transform(x_test)

In [15]:
model = Sequential()
#model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2]), bias_regularizer=L1L2(l1=0.01, l2=0.01)))
#model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(2000, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.2, noise_shape=None, seed=None))
model.add(BatchNormalization())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
history = model.fit([x_train], batch_size=100, y=y_train, verbose=1, validation_data=(x_valid, y_valid), 
          shuffle=True, epochs=20)

Instructions for updating:
Use tf.cast instead.
Train on 310746 samples, validate on 28020 samples
Epoch 1/20
  4200/310746 [..............................] - ETA: 1:45 - loss: 1.5700 - acc: 0.5400